## Fly song notebook

The goal is to identify two distinct motifs in recordings of Drosophila song: *pulse* song and *sine* song.

Although there exist __[complex strategies](https://bmcbiol.biomedcentral.com/articles/10.1186/1741-7007-11-11)__ for performing this analysis, we will use a very simple approach here.

For detection of *pulse song*, we will threshold the audio signal, under the assumption that the magnitude of bouts of pulse song is consistently higher than that of sine song or no song.

For detection of *sine song*, we will compute a short-time Fourier transform (STFT) and evaluate the peak spectral power (And associated frequency) at each time bin.

For both types of song, we then filter bouts to be of a minimum duration.

In [ ]:
# Import packages

import neo
import matplotlib.pyplot as pl
from scipy.ndimage import label
from scipy.signal import stft
%matplotlib notebook

In [ ]:
# define a function to plot a line coloured by another variable
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D

def label_plot(x, y, c, c_colors=None, labels=None, ax=None, **kwargs):
    """Plot y versus x, colored c_color when c is nonzero
    """
    x = np.asarray(x).astype(float)
    y = np.asarray(y).astype(float)
    c = np.asarray(c).astype(int)
    ucol = np.unique(c)
    ucol = ucol[ucol!=0]
    
    if ax is None:
        ax = pl.gca()
    cmap = kwargs.pop('cmap', pl.cm.viridis)
    if c_colors is None:
        c_colors = cmap(np.linspace(0,1,len(ucol)))
    
    color = kwargs.pop('color', 'k')
    color = mcolors.to_rgba(color)
    
    lcmap = ListedColormap(np.concatenate([[color], c_colors]))
    norm = BoundaryNorm(np.arange(lcmap.N+1), lcmap.N)
    
    points = np.array([x, y]).T[:,None,:]
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = LineCollection(segments, cmap=lcmap, norm=norm)
    lc.set_array((c[1:] | c[:-1]))
    
    coll = ax.add_collection(lc, **kwargs)
    ax.axis('auto')
    
    # legend
    if labels is not None:
        legend_lines = [Line2D([0], [0], color=lcmap.colors[i], lw=4) for i in range(lcmap.N)]
        ax.legend(legend_lines, labels, loc='best')
    
    return ax

In [ ]:
## -- User parameters -- ##

# data parameters
data_file = 'flysong.abf' # path to data file of interest

# analysis parameters
stft_window = .025 # length of STFT window in seconds
min_sine_dur = .100 # minimum continuous time window for a sine song bout in seconds
min_pulse_dur = .100 # minimum continuous time window for a pulse song bout in seconds
within_pulse_gap = .150 # maximum duration of time in seconds allowed below threshold inside a single pulse bout
pulse_magnitude_thresh = 3.8 # number of absolute standard deviations from mean to consider a pulse signal

In [ ]:
# load the data file into Python
r = neo.AxonIO(filename=data_file)
block = r.read_block()
segment = block.segments[0]

data = segment.analogsignals[0]
Ts = float(data.sampling_period)
fs = 1/Ts
data = data.as_array().squeeze()

In [ ]:
# process the parameters
win_ = int(round(stft_window/Ts)) # length of STFT window in samples
min_pulse_dur_ = int(round(min_pulse_dur/Ts)) # minimum continuous time window for a pulse song bout in samples
within_pulse_gap_ = int(round(within_pulse_gap/Ts)) # maximum duration of time in samples allowed below threshold inside a single pulse bout

In [ ]:
## -- Pulse song -- ##

# threshold
putative_pulse = np.abs(data) > data.mean() + pulse_magnitude_thresh*data.std()

# fill gaps within tolerance
neg_labs,_ = label(~putative_pulse)
gaps_to_fill = np.array([l for l in np.unique(neg_labs) if np.sum(neg_labs==l)<=within_pulse_gap_])
fill = np.array([nl in gaps_to_fill for nl in neg_labs])

is_pulse = putative_pulse | fill

In [ ]:
## -- Sine song -- ##

# perform STFT
freq,t,z = stft(data, fs=1/Ts, nperseg=win_, window='hann')
stft_Ts = np.mean(np.diff(t))
min_sine_dur_ = int(round(min_sine_dur/stft_Ts)) # minimum continuous time window for a sine song bout in samples
power = np.abs(z)**2 # perhaps just magnitude and not power will be better, not clear yet

# display STFT
#pl.pcolor(t, freq, power)

# analyze STFT

# power magnitude criteria
thresh = np.median(power)
peak_power = np.max(power, axis=0)
is_high = peak_power > thresh

# freqency criteria
peak_freq = freq[np.argmax(power, axis=0)]
is_band = (peak_freq>80) & (peak_freq<220)

# combine magnitude and frequency critera
putative_sine = is_high & is_band

# threshold continuous segments that pass criteria
labs,nlab = label(putative_sine)
keep_labs = np.array([l for l in np.unique(labs) if np.sum(labs==l)>=min_sine_dur_ and l!=0])
labs = np.array([True if l in keep_labs else False for l in labs])
is_sine = np.repeat(labs, int(round(data.size/labs.size)))[:len(data)]

is_sine = is_sine & (~is_pulse) # sine song is only possible if a timepoint is not already considered pulse song

In [ ]:
time = np.arange(len(data))*Ts

song = np.zeros_like(data)
song[is_pulse] = 1
song[is_sine] = 2

ax = label_plot(time, data, song, labels=['No song','Pulse song','Sine song'], cmap=pl.cm.winter)
ax.set_xlabel('Time (sec)')